In [20]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt 
import torch 
import torch.nn as nn

import warnings
warnings.filterwarnings("ignore")

In [21]:
y_pred = pd.read_csv('../task3/clean_data/clean_pred.csv')
y_true = pd.read_csv('../task3/clean_data/clean_true.csv')
rf = pd.read_csv('../task3/clean_data/clean_ortho_rf.csv')
factor_ret = pd.read_csv('../task3/clean_data/clean_factor_ret.csv')

该notebook主要实现y_pred,y_true,rf,factor_ret表的数据对齐(即每个trade_date都有相同维度的数据）

In [22]:
# 删除y_true表中数据不足339天的股票和它们在剩下三张表的数据
code_delete = []
code_idx = np.sort(np.array(list(set(y_true['stock_code']))))
for code in code_idx:
    x = y_true[y_true['stock_code']==code]
    if len(x)<339:
        code_delete.append(code)
        y_true.drop(y_true[y_true['stock_code']==code].index,inplace=True)
        y_pred.drop(y_pred[y_pred['stock_code']==code].index,inplace=True)
        rf.drop(rf[rf['stock_code']==code].index,inplace=True)

In [23]:
# 删除y_pred表中不足339天的股票和它们在y_pred表和rf表的数据(这是为了保证每天输入的特征的维数相等)
code_delete2 =[]
code_idx = np.sort(np.array(list(set(code_idx)-set(code_delete))))
for code in code_idx:
    x = y_pred[y_pred['stock_code']==code]
    if len(x)<339:
        code_delete2.append(code)
        y_true.drop(y_true[y_true['stock_code']==code].index,inplace=True)
        y_pred.drop(y_pred[y_pred['stock_code']==code].index,inplace=True)
        rf.drop(rf[rf['stock_code']==code].index,inplace=True)

In [27]:
# rf表缺失trade_date=3616的数据，可通过线性插值（取前一天和后一天对应risk_factor的平均值）的方法填充
code_idx = np.sort(np.array(list(set(code_idx)-set(code_delete2))))
df = pd.DataFrame({'trade_date':[3616]*len(code_idx),'stock_code':len(code_idx)})
for factor in list(rf.columns)[2:]:
    x = pd.DataFrame({factor:(rf[rf['trade_date']==3615][factor].values+rf[rf['trade_date']==3617][factor].values)/2})
    df = pd.concat([df,x],axis=1)

rf = pd.concat([rf,df],axis=0)

In [28]:
# factor_ret表缺失trade_date=3616的数据，可通过线性插值（取前一天和后一天对应risk_factor的平均值）的方法填充
factor_list = list(factor_ret.columns)[1:]
temp = (factor_ret[factor_ret['trade_date']==3615].iloc[0,1:].values+factor_ret[factor_ret['trade_date']==3617].iloc[0,1:].values)/2
df = pd.DataFrame({'trade_date':[3616]})
for i,factor in enumerate(factor_list):
    x = pd.DataFrame({factor:[temp[i]]})
    df = pd.concat([df,x],axis=1)

factor_ret = pd.concat([factor_ret,df],axis=0)

In [32]:
y_true.to_csv('../task3/clean_data/clean_true_process.csv',index=False)
y_pred.to_csv('../task3/clean_data/clean_pred_process.csv',index=False)
rf.to_csv('../task3/clean_data/clean_ortho_rf_process.csv',index=False)
factor_ret.to_csv('../task3/clean_data/clean_factor_ret_process.csv',index=False)